# Theano 配置和编译模式

## 配置

之前我们已经知道， `theano` 的配置可以用 `config` 模块来查看：

In [1]:
import theano
import theano.tensor as T

print theano.config

floatX (('float64', 'float32', 'float16')) 
    Doc:  Default floating-point precision for python casts.

Note: float16 support is experimental, use at your own risk.
    Value:  float32

warn_float64 (('ignore', 'warn', 'raise', 'pdb')) 
    Doc:  Do an action when a tensor variable with float64 dtype is created. They can't be run on the GPU with the current(old) gpu back-end and are slow with gamer GPUs.
    Value:  ignore

cast_policy (('custom', 'numpy+floatX')) 
    Doc:  Rules for implicit type casting
    Value:  custom

int_division (('int', 'raise', 'floatX')) 
    Doc:  What to do when one computes x / y, where both x and y are of integer types
    Value:  int

device (cpu, gpu*, opencl*, cuda*) 
    Doc:  Default device for computations. If gpu*, change the default to try to move computation to it and to put shared variable of float32 on it. Do not use upper case letters, only lower case even if NVIDIA use capital letters.
    Value:  gpu1

init_gpu_device (, gpu*, opencl*, 

Using gpu device 1: Tesla K10.G2.8GB (CNMeM is disabled)


这些配置影响着 `theano` 的运行，很多的参数都是只读的，因此，**我们应当尽量避免在程序中直接修改这些参数**。

大部分参数都有指定的默认值，我们可以在 `.theanorc` 文件中对配置进行修改，也可以在环境变量 `THEANO_FLAGS` 中进行修改，它们的优先级顺序如下：

- 首先是对 `theano.config.<property>` 的赋值
- 然后是 `THEANO_FLAGS` 环境变量指定的内容
- 最后是 `.theanorc` 文件或者 `THEANORC` 环境变量所指示的文件中的内容

具体的参数含义可以参考：

http://deeplearning.net/software/theano/library/config.html

### 环境变量 THEANO_FLAGS 

使用 `THEANO_FLAGS` 环境变量，运行程序的方法如下：

    THEANO_FLAGS='floatX=float32,device=gpu0,nvcc.fastmath=True'  python <myscript>.py
    
如果是 `window` 下，则需要进行稍微的改动：

    set THEANO_FLAGS='floatX=float32,device=gpu0,nvcc.fastmath=True' && python <myscript>.py
    
示例中的配置将浮点数的精度设为了 `32` 位，并将使用 `GPU 0` 和 `CUDA` 的 `fastmath` 模式进行编译和运算。

### 配置文件 THEANORC

环境变量 `THEANORC` 的默认位置为 `$HOME/.theanorc` （`windows` 下为 `$HOME/.theanorc:$HOME/.theanorc.txt`）。

与前面 `THEANO_FLAGS` 指定的内容相同的配置文件为：

    [global]
    floatX = float32
    device = gpu0

    [nvcc]
    fastmath = True



这里 `[golbal]` 对应的是 `config` 中的参数，如 `config.device, config.mode`； `config` 的子模块中的参数，如 `config.nvcc.fastmath, config.blas.ldflags` 则需要用 `[nvcc], [blas]` 的部分去设置。

## 模式

每次调用 `theano.function` 的时候，那些符号变量之间的结构会被优化和计算，而优化和计算的模式都是由 `config.mode` 所决定的。

`Theano` 中定义了这四种模式：

- `FAST_COMPILE`
    - `compile.mode.Mode(linker='py', optimizer='fast_compile')`
    - `Python` 实现，构造很快，运行慢
- `FAST_RUN`
    - `compile.mode.Mode(linker='cvm', optimizer='fast_run')`
    - `C` 实现，构造较慢，运行快
- `DebugMode`
    - `compile.debugmode.DebugMode()`
    - 调试模式，两种实现都可以
- `ProfileMode`
    - `compile.profilemode.ProfileMode()`
    - `C` 实现，已经停用，使用 `theano.profile` 替代
    
更多的细节，可以参考：

http://deeplearning.net/software/theano/library/compile/mode.html#libdoc-compile-mode

### Linkers

从上面的定义可以看出，一个模式由两部分构成，`optimizer` 和 `linker`， `ProfileMode` 和 `DebugMode` 模式使用自带的 `linker`。

可用的 `linker` 可以从下表中查看：

http://deeplearning.net/software/theano/tutorial/modes.html#linkers

### 使用 DebugMode

一般在使用 `FAST_RUN` 或者 `FAST_COMPILE` 模式之前，最好先用 `DebugMode` 进行调试，不过速度会比前两个模式慢得多。

我们用一个实例看一下两者的区别：

In [2]:
x = T.dvector('x')

f_1 = theano.function([x], 10 / x)

print f_1([5])
print f_1([0])
print f_1([7])

[ 2.]
[ inf]
[ 1.42857143]


在非 Debug 模式下，除以 0 是合法的，但是在 `DebugMode` 下，会给出错误，帮助我们进行调试：

In [3]:
f_2 = theano.function([x], 10 / x, mode='DebugMode')

print f_2([5])
print f_2([0])
print f_2([7])

[ 2.]


InvalidValueError: InvalidValueError
        type(variable) = TensorType(float64, vector)
        variable       = Elemwise{true_div,no_inplace}.0
        type(value)    = <type 'numpy.ndarray'>
        dtype(value)   = float64
        shape(value)   = (1,)
        value          = [ inf]
        min(value)     = inf
        max(value)     = inf
        isfinite       = False
        client_node    = None
        hint           = perform output
        specific_hint  = non-finite elements not allowed
        context        = ...
  Elemwise{true_div,no_inplace} [id A] ''   
   |TensorConstant{(1,) of 10.0} [id B]
   |x [id C]

        

更多细节可以参考：

http://deeplearning.net/software/theano/library/compile/debugmode.html#debugmode